## Table of contents

- [Imports](#im)
- [Data and Setup](#1) 
- [Baselines](#2) 
- [Collaborative filtering](#3) 
- [Content-based recommendations](#4) 


## Imports <a name="im"></a>

In [2]:
import os
import numpy as np
import pandas as pd

#plotting
import matplotlib.pyplot as plt
import plotly.express as px

#sklearn
from sklearn.decomposition import PCA
from sklearn.model_selection import cross_validate, train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import KNNImputer
from sklearn.linear_model import Ridge

#surprise package
import surprise
from surprise import SVD, Dataset, Reader, accuracy
from surprise.model_selection import cross_validate

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

## Data and Setup <a name="1"></a>
<hr>


In [3]:
#read in data 
r_cols = ["user_id", "movie_id", "rating", "timestamp"]
ratings = pd.read_csv(
    "../ml-100k/u.data",
    sep="\t",
    names=r_cols,
    encoding="latin-1",
)
ratings.head()

,user_id,movie_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [4]:
#plot
fig = px.bar(ratings["rating"].value_counts(normalize=True),
            labels={'value':'Count of records (% of total)', 'index': 'Movie Rating'},
            width=800, height=400,
            title = "Breakdown of movie ratings count")
fig.update_layout(showlegend=False)
fig.show()

In [5]:
# get number of users and movies in the data
user_key = "user_id"
item_key = "movie_id"

N = len(ratings[user_key].unique())
M = len(ratings[item_key].unique())

print(f'Number of users (N): {N}')
print(f'Number of movies (M): {M}')

Number of users (N): 943
Number of movies (M): 1682


In [6]:
print(f'Fraction of non missing ratings in Utility Matrix Y: {round(len(ratings) / (N * M), 3)}')

Fraction of non missing ratings in Utility Matrix Y: 0.063


In [7]:
print(f'Average number of ratings per user: {round(len(ratings) / N)}')
print(f'Average number of ratings per movie: {round(len(ratings) / M)}')

Average number of ratings per user: 106
Average number of ratings per movie: 59


In [8]:
#split data into train test splits
X = ratings.copy()
y = ratings[user_key]
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

X_train.shape, X_valid.shape

((80000, 4), (20000, 4))

<br>

### Creating the  Utility Matrix 

In [9]:
#create utility matrices for train and validation sets 
user_mapper = dict(zip(np.unique(ratings[user_key]), list(range(N))))
item_mapper = dict(zip(np.unique(ratings[item_key]), list(range(M))))
# user_inverse_mapper = dict(zip(list(range(N)), np.unique(ratings[user_key])))
# item_inverse_mapper = dict(zip(list(range(M)), np.unique(ratings[item_key])))

# helper function to create utility matrix
def create_Ymatrix_from_ratings(data, N, M):
    """
    function creates matrix from ratings df
    """
    Y = np.zeros((N, M))
    Y.fill(np.nan)
    for index, val in data.iterrows():
        n = user_mapper[val[user_key]]
        m = item_mapper[val[item_key]]
        Y[n, m] = val["rating"]

    return Y


In [10]:
# create train and validation matrices using function
train_mat = create_Ymatrix_from_ratings(X_train, N, M)
valid_mat = create_Ymatrix_from_ratings(X_valid, N, M)

In [11]:
print(f'Shape of train_mat N x M: {train_mat.shape}')
print(f'Shape of valid_mat N x M: {valid_mat.shape}')

Shape of train_mat N x M: (943, 1682)
Shape of valid_mat N x M: (943, 1682)


> `train_mat` only has the ratings from X_train whereas `valid_mat` only has ratings from X_valid

<br>

### Evaluation Metric


The evaluation metric used to measure the performance of various recommender systems will be RMSE (Root mean square error). 
RMSE calculates the error between the actual ratings and the predicted ratings.

In [12]:
# helper functions to calculate RMSE 
def error(Y1, Y2):
    """
    Returns the root mean squared error (RMSE).
    """
    return np.sqrt(np.nanmean((Y1 - Y2) ** 2))


def evaluate(pred_Y, train_mat, valid_mat, model_name="Global average"):
    """
    Evaluates the train and validation RMSEs
    """
    print("%s train RMSE: %0.2f" % (model_name, error(pred_Y, train_mat)))
    print("%s valid RMSE: %0.2f" % (model_name, error(pred_Y, valid_mat)))

## Exploring Baselines <a name="2"></a>
<hr>

**Global average rating baseline**

In [13]:
#predict every rating as the global average
global_avg = np.nanmean(train_mat)
pred_g = np.zeros(train_mat.shape) + global_avg

#evaluate
evaluate(pred_g, train_mat, valid_mat, model_name="Global average")

Global average train RMSE: 1.13
Global average valid RMSE: 1.12


**Per-user average rating baseline**

In [14]:
#predict every rating as the user average for each user
user_avg = np.nanmean(train_mat, axis=1)
#user_avg[np.isnan(user_avg)] = avg
pred_n = np.tile(user_avg[:, None], (1, M))

# evaluate
evaluate(pred_n, train_mat, valid_mat, model_name="Per-user average")

Per-user average train RMSE: 1.03
Per-user average valid RMSE: 1.04


**Per-movie average rating baseline**

In [15]:
#predict every rating as the movie average for each movie
movie_avg = np.nanmean(train_mat, axis=0)
#avg_m[np.isnan(avg_m)] = avg
pred_m = np.tile(movie_avg[None, :], (N,1))

evaluate(pred_m, train_mat, valid_mat, model_name="Per-movie average")

Per-movie average train RMSE: 1.00
Per-movie average valid RMSE: 1.02


<ipython-input-15-6577eb93dfbb>:2: RuntimeWarning:

Mean of empty slice



**Per-user and per-movie average rating**

In [16]:
#predict every rating as the per-user and per-movie average
pred_nm = (user_avg[:, None] + movie_avg[None, :])/2
evaluate(pred_nm, train_mat, valid_mat, model_name = "per-movie and per-user average")

per-movie and per-user average train RMSE: 0.96
per-movie and per-user average valid RMSE: 0.98


**K-Nearest Neighbours imputation for ratings**

In [17]:
#remove columns where all entries are NaN 
knn_train_mat = train_mat[:,~np.all(np.isnan(train_mat), axis=0)]
knn_valid_mat = valid_mat[:,~np.all(np.isnan(train_mat), axis=0)]

imputer = KNNImputer(n_neighbors=20)
knn_preds = imputer.fit_transform(knn_train_mat)

# evaluate
evaluate(knn_preds, knn_train_mat, knn_valid_mat, model_name = "Knn")

Knn train RMSE: 0.00
Knn valid RMSE: 0.97


> KNN train has RMSE: 0.00 because KNNImpter is only filling in missing values, hence the perfect score

## Collaborative filtering <a name="3"></a>


Using [`surprise`](https://surprise.readthedocs.io/en/stable/) package's implementation of SVD for recommendation systems

In [18]:
#read in data and get train validation splits 
ratings_drop = ratings.drop(columns = "timestamp")

reader = Reader()
data = Dataset.load_from_df(ratings_drop, reader)  # Load the data

trainset, validset = surprise.model_selection.train_test_split(data, test_size=0.2, random_state=42) 

In [19]:
# train model and evaluation
k=10 #reduce number of dimensions to k 
s_svd = SVD(n_factors=k, random_state=42)
s_svd.fit(trainset)
s_svd_preds = s_svd.test(validset)

results = cross_validate(s_svd, data, measures=["RMSE", "MAE"], cv=5, verbose=True)
pd.DataFrame(results).mean()

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9412  0.9323  0.9313  0.9295  0.9423  0.9353  0.0053  
MAE (testset)     0.7449  0.7353  0.7351  0.7336  0.7437  0.7385  0.0048  
Fit time          1.70    1.71    1.70    1.74    1.67    1.70    0.02    
Test time         0.12    0.11    0.13    0.25    0.11    0.14    0.06    


test_rmse    0.935335
test_mae     0.738545
fit_time     1.704273
test_time    0.143396
dtype: float64

<br>

### Getting top *n* predictions for users

In [20]:
#fit model
trainset, validset = surprise.model_selection.train_test_split(
    data, test_size=0.2, random_state=42
)

k = 10
algo = SVD(n_factors=k, random_state=42)
algo.fit(trainset)
svd_preds = algo.test(validset)
accuracy.rmse(svd_preds, verbose=True)

RMSE: 0.9328


0.932839700199596

In [21]:
# Attributions: Functions below from Surprise package:
# https://github.com/NicolasHug/Surprise/blob/master/examples/top_n_recommendations.py

from collections import defaultdict

def get_top_n(predictions, n=10):
    """Return the top-N recommendation for each user from a set of predictions.

    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        n(int): The number of recommendation to output for each user. Default
            is 10.

    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n.
    """

    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n


def top_n_recs(user_id, n=5):
    top_n = get_top_n(svd_preds, n=n)
    return pd.DataFrame(top_n[user_id], columns=["movie_id", "pred"])


In [22]:
# get top 5 recommendations for 5 random users 
user_id_sample = ratings["user_id"].sample(5).to_list()
n = 5
for user_id in user_id_sample:
    print("\nTop %d recommendations for user %d" % (n, user_id))
    print(top_n_recs(user_id))



Top 5 recommendations for user 390
   movie_id      pred
0       302  4.416016
1       181  4.353251
2       275  4.259580
3       475  4.221498
4       286  3.992420

Top 5 recommendations for user 262
   movie_id      pred
0        50  4.063574
1       191  3.927542
2       185  3.844298
3       195  3.767582
4       432  3.694664

Top 5 recommendations for user 589
   movie_id      pred
0       336  3.010154

Top 5 recommendations for user 940
   movie_id      pred
0        98  4.055584
1       657  4.019260
2       474  3.964896
3       285  3.927178
4       315  3.840518

Top 5 recommendations for user 3
   movie_id      pred
0       344  3.248262
1       303  3.136777
2       300  3.088792
3       320  3.073646
4       341  2.943108


<br><br>

## Content-based recommenders <a name="4"></a>
<hr> 


### 1. Reading in `u.item` Data

In [25]:
# read in data for u.item (movie )
cols = [
    "movie_id",
    "movie title",
    "release date",
    "video release date",
    "IMDb URL",
    "unknown",
    "Action",
    "Adventure",
    "Animation",
    "Children",
    "Comedy",
    "Crime",
    "Documentary",
    "Drama",
    "Fantasy",
    "Film-Noir",
    "Horror",
    "Musical",
    "Mystery",
    "Romance",
    "Sci-Fi",
    "Thriller",
    "War",
    "Western",
]

movies_data = pd.read_csv(
    "../ml-100k/u.item",
    sep="|",
    names=cols,
    encoding="latin-1",
)


In [26]:
#take only the movie genre attributes
genres = [
    "Action",
    "Adventure",
    "Animation",
    "Children",
    "Comedy",
    "Crime",
    "Documentary",
    "Drama",
    "Fantasy",
    "Film-Noir",
    "Horror",
    "Musical",
    "Mystery",
    "Romance",
    "Sci-Fi",
    "Thriller",
    "War",
    "Western",
]
movie_genres = movies_data[genres]
movie_genres.head()

,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0


In [27]:
Z = movie_genres.to_numpy()
Z.shape

(1682, 18)

In [28]:
print("Average number of genres per movie: %.1f" % (Z.sum() / M))

Average number of genres per movie: 1.7


<br>

### 2. Setting up data for regression

In [29]:
from collections import defaultdict

# function creates X and y for each user for a supervised learning approach
def get_X_y_per_user(ratings_df, d=Z.shape[1]):
    """
    Returns X and y for each user.

    Parameters:
    ----------
    ratings_df : pandas.DataFrame
         ratings data as a dataframe

    d : int
        number of item features

    Return:
    ----------
        dictionaries containing X and y for all users
    """
    lr_y = defaultdict(list)
    lr_X = defaultdict(list)

    for index, val in ratings_df.iterrows():
        n = user_mapper[val[user_key]]
        m = item_mapper[val[item_key]]
        lr_X[n].append(Z[m])
        lr_y[n].append(val["rating"])

    for n in lr_X:
        lr_X[n] = np.array(lr_X[n])
        lr_y[n] = np.array(lr_y[n])

    return lr_X, lr_y

In [30]:
# call function and get X and y train/valid for each user
X_train_usr, y_train_usr = get_X_y_per_user(X_train);
X_valid_usr, y_valid_usr = get_X_y_per_user(X_valid);

In [31]:
# sanity check - users will have different number of rows (movies) because rows represent only movies the user has rated
print("User 1:")
print(f'Shape of X_train_usr for one user (movies x genres): {pd.DataFrame(X_train_usr[1]).shape}')
print(f'Shape of y_train_usr for one user (movies x rating): {pd.DataFrame(y_train_usr[1]).shape}')
print("")
print("User 25:")
print(f'Shape of X_train_usr for another user (movies x genres): {pd.DataFrame(X_train_usr[25]).shape}')
print(f'Shape of y_train_usr for another user (movies x user rating): {pd.DataFrame(y_train_usr[25]).shape}')

User 1:
Shape of X_train_usr for one user (movies x genres): (46, 18)
Shape of y_train_usr for one user (movies x rating): (46, 1)

User 25:
Shape of X_train_usr for another user (movies x genres): (85, 18)
Shape of y_train_usr for another user (movies x user rating): (85, 1)


<br>

### 3. Train model and Predict Ratings

In [32]:
# functions to train and predict ratings
def train_usr(user_name, model=Ridge()):
    """
    train model; default uses Ridge Regression
    """
    X = X_train_usr[user_name] 
    y = y_train_usr[user_name] 
    model.fit(X, y)
    return model


def predict_usr(model):
    """
    Predict ratings for movies
    """
    feat_vecs = movie_genres
    preds = model.predict(feat_vecs)
    return preds

In [33]:
#train and predict for one user 
model_1 = train_usr(0)
preds = predict_usr(model_1)
recon_x = pd.DataFrame(preds)

#prediction for user 0
recon_x.head()

c:\Users\Jegadit\anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning:

X has feature names, but Ridge was fitted without feature names



,0
0,2.399218
1,2.863537
2,3.549176
3,3.892106
4,3.755807


In [34]:
#train and predict for all users 
users = range(1, train_mat.shape[0])

for i in users:
    model = train_usr(i)
    scores = predict_usr(model)
    recon_x[i] = pd.DataFrame(scores)

c:\Users\Jegadit\anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning:

X has feature names, but Ridge was fitted without feature names

c:\Users\Jegadit\anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning:

X has feature names, but Ridge was fitted without feature names

c:\Users\Jegadit\anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning:

X has feature names, but Ridge was fitted without feature names

c:\Users\Jegadit\anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning:

X has feature names, but Ridge was fitted without feature names

c:\Users\Jegadit\anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning:

X has feature names, but Ridge was fitted without feature names

c:\Users\Jegadit\anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning:

X has feature names, but Ridge was fitted without feature names

c:\Users\Jegadit\anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning:

X has feature names, but Ridge was fitted without 

In [35]:
# prediction for all users and movies in the utility matrix
recon_x.T.head()

,0,1,2,3,4,5,6,7,8,9,...,1672,1673,1674,1675,1676,1677,1678,1679,1680,1681
0,2.399218,2.863537,3.549176,3.892106,3.755807,4.043851,4.691223,2.828793,4.043851,3.880620,...,3.325411,4.043851,4.043851,4.043851,4.043851,4.043851,3.870601,4.365276,3.629682,4.043851
1,3.478037,4.368664,3.162929,4.699790,3.860238,3.819291,3.459575,3.703311,3.819291,3.940570,...,3.550115,3.819291,3.819291,3.819291,3.819291,3.819291,3.612328,4.268690,3.565369,3.819291
2,2.513182,2.525054,2.101808,3.048204,3.000602,2.934748,2.714180,2.732854,2.934748,2.909098,...,2.417158,2.934748,2.934748,2.934748,2.934748,2.934748,2.633232,3.466172,2.513182,2.934748
3,5.031081,3.461739,4.403304,4.966239,4.753188,4.818288,4.484910,5.214774,4.818288,4.818288,...,4.154768,4.818288,4.818288,4.818288,4.818288,4.818288,4.022303,4.437287,5.031081,4.818288
4,3.459727,2.511443,2.216249,3.459372,3.918273,2.736089,3.540800,2.591529,2.736089,2.903190,...,2.330173,2.736089,2.736089,2.736089,2.736089,2.736089,1.403143,1.922983,2.977611,2.736089


<br>

### 4. Evaluating Results

In [36]:
#evaluate content-based method
evaluate(recon_x.T, train_mat, valid_mat, model_name="Content-Based filtering")

Content-Based filtering train RMSE: 0.91
Content-Based filtering valid RMSE: 1.05


<br><br>